# Grounding Generative AI using Enterprise Search Results

_This colab demonstrates examples of Retrieval Augmented Generation (RAG)
and Reasoning + Acting (ReAct) workflows that make use of Google Enterprise Search (ES) and Large Language Models (LLMs). Langchain is also used for many Quality of Life features. Please make a separate copy of this notebook before making any changes._


* _**Credit to** `rthallam@` for their great work on langchain integration [generative-ai Github repo](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/language/examples/langchain-intro)_

* _**Self link**: [go/genai-colab-es-palm-langchain](go/colab-langchain-vertex-genai)_
* _**Author**: `tompakeman@`_
* _**Last Update**: June 8th 2023_
* _**Context**: [RAG Arxiv Paper](https://arxiv.org/pdf/2005.11401.pdf) & [ReACT Arxiv Paper](https://arxiv.org/abs/2210.03629.pdf)_


### Overview

**LLMs**
* Google Cloud's PaLM model and other generative large language models (LLMs) provide a powerful way to generate new data such as text, image and code.
* However, LLMs are probabilistic systems which do not possess real 'understanding' - only 'parametric' knowledge encoded in the weighting of relationships between entities in their training data. As such they are susceptible to 'hallucination' whereby they confidently provide answers to user questions which are factually inaccurate.

**Retrieval Augmented Generation**
* Retrieval Augmented Generation (RAG) is a technique which attempts to mitigate the problem of hallucination. In RAG factual information (also called non-parametric data) is retrieved from a data store or search engine and then inserted into the prompt which is sent to the LLM.
* The LLM is asked to summarize the information in the prompt rather than generate its own response. This requires careful prompt design, but when successful it can help 'ground' the LLM response in factual, proprietary data.

**Enterprise Search**
* 'Enterprise Search' is a Google Cloud solution which combines LLM with Google search technology to provide users with an out of the box search engine solution
* Search engines can be configured to ingest and index structured, unstructured or website data and then retrieve information from this data using natural language queries.
* LLMs are used to provide summarization and conversational abilities.
* In this notebook, Enterprise Search is used to retrieve data from a search engine, which is then summarized using Google's PaLM LLM.
---

### Technical Notes
* This uses Colab's inbuilt OAuth flow for authentication, so your Google account will need to have access to a GCP project with the ES + LLM APIs enabled in order for this to work.
  * The permissions required are "Discovery Engine Admin" and "Vertex AI Administrator". If running this in Vertex Workbench you will need to grant these permissions to the Service Account that is used by Workbench
* No additional fine-tuning, prompt-tuning or retraining has been done on any of the models used here
* The examples used here are from private Google projects and search engines - you will need to replace them with your own values.

In [ ]:
from google.colab import auth
from google.auth import default

auth.authenticate_user()
creds, _ = default()

# Installation
**NOTE: This requires shutting down the runtime.**

In [ ]:
# Install langchain
! pip install langchain --upgrade

# Install Enterprise Search SDK and Vertex PaLM endpoint
! pip install google_cloud_discoveryengine

# Install Google Cloud Platform
! pip install google-cloud-aiplatform --upgrade

import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.1/431.1 kB 7.7 MB/s eta 0:00:00


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.6 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.1
    Uninstalling shapely-2.0.1:
      Successfully uninstalled shapely-2.0.1


{'status': 'ok', 'restart': True}

# Config

In [ ]:
#@title ### You will need to update these values
VERTEX_API_PROJECT = 'your-project' #@param {"type": "string"}
VERTEX_API_LOCATION = 'us-central1' #@param {"type": "string"}

# Imports and classes

In [ ]:
from google.cloud import discoveryengine_v1beta
from google.protobuf.json_format import MessageToDict

import vertexai
vertexai.init(project=VERTEX_API_PROJECT, location=VERTEX_API_LOCATION)

from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings

from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.base import Chain
from langchain.chains.question_answering import load_qa_chain
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool
from langchain.callbacks.manager import Callbacks, CallbackManagerForChainRun
from langchain.schema import AgentAction, AgentFinish, Document

from typing import Any, Mapping, List, Dict, Optional, Tuple, Sequence, Union
import json, re

In [ ]:
#@title Additional Enterprise Search Classes and helper functions

class EnterpriseSearchRetriever():
  """Retriever class to fetch documents or snippets from a search engine."""
  def __init__(self,
               project,
               search_engine,
               location='global',
               serving_config_id='default_config'):
    self.search_client = discoveryengine_v1beta.SearchServiceClient()
    self.serving_config: str = self.search_client.serving_config_path(
            project=project,
            location=location,
            data_store=search_engine,
            serving_config=serving_config_id,
            )

  def _search(self, query:str):
    """Helper function to run a search"""
    request = discoveryengine_v1beta.SearchRequest(serving_config=self.serving_config, query=query)
    return self.search_client.search(request)

  def get_relevant_documents(self, query: str) -> List[Document]:
    """Retrieve langchain Documents from a search response"""
    res = self._search(query)
    documents = []
    for result in res.results:
        data = MessageToDict(result.document._pb)
        metadata = data.copy()
        if 'derivedStructData' in metadata:
            del metadata['derivedStructData']
        if 'structData' in metadata:
            del metadata['structData']
        if data.get('derivedStructData') is None:
            content = json.dumps(data.get('structData', {}))
        else:
            content = json.dumps([d.get('snippet') for d in data.get('derivedStructData', {}).get('snippets', []) if d.get('snippet') is not None])
        documents.append(Document(page_content=content, metadata=metadata))
    return documents

  def get_relevant_snippets(self, query: str) -> List[str]:
    """Retrieve snippets from a search query"""
    res = self._search(query)
    snippets = []
    for result in res.results:
        data = MessageToDict(result.document._pb)
        if data.get('derivedStructData', {}) == {}:
            snippets.append(json.dumps(data.get('structData', {})))
        else:
            snippets.extend([d.get('snippet') for d in data.get('derivedStructData', {}).get('snippets', []) if d.get('snippet') is not None])
    return snippets


class EnterpriseSearchChain(Chain):
    """Chain that queries an Enterprise Search Engine and summarizes the responses."""

    chain: Optional[LLMChain]
    search_client: Optional[EnterpriseSearchRetriever]

    def __init__(self,
                 project,
                 search_engine,
                 chain,
                 location='global',
                 serving_config_id='default_config'):
        super().__init__()
        self.chain = chain
        self.search_client = EnterpriseSearchRetriever(project, search_engine, location, serving_config_id)

    @property
    def input_keys(self) -> List[str]:
        return ['query']

    @property
    def output_keys(self) -> List[str]:
        return ['summary']

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        _run_manager = CallbackManagerForChainRun.get_noop_manager()
        query = inputs['query']
        _run_manager.on_text(query, color="green", end="\n", verbose=self.verbose)
        snippets = self.search_client.get_relevant_snippets(query)
        _run_manager.on_text(snippets, color="white", end="\n", verbose=self.verbose)
        summary = self.chain.run(snippets)
        return {'summary': summary}


    @property
    def _chain_type(self) -> str:
        return "google_enterprise_search_chain"

# Pattern 1 - A simple example of retreiving information from search and summarizing it
_In this example, we use a search engine containing Alphabet Investor PDFs (an unstructured Enterprise Search engine). We retrieve a set of search results (snippets from individual PDF documents) and then pass these into an LLM prompt. We ask the LLM to summarize the results_

#### Use Cases
* Retrieving and summarizing data that exists across various sources
* Structuring unstructured data, e.g. converting financial data stored in PDFs to a Pandas dataframe

In [ ]:
#@title Initialise the LLM
GCP_PROJECT = VERTEX_API_PROJECT
SEARCH_ENGINE = "your-engine-id" #@param {type: "string"}
LLM_MODEL = "text-bison" #@param {type: "string"}
MAX_OUTPUT_TOKENS = 1024 #@param {type: "integer"}
TEMPERATURE = 0.2 #@param {type: "number"}
TOP_P = 0.8 #@param {type: "number"}
TOP_K = 40 #@param {type: "number"}
VERBOSE = True #@param {type: "boolean"}
llm_params = dict(
    model_name=LLM_MODEL,
    max_output_tokens=MAX_OUTPUT_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P,
    top_k=TOP_K,
    verbose=VERBOSE,
)

llm = VertexAI(**llm_params)

In [ ]:
#@title Example - summarize financial results
SEARCH_QUERY = 'Total Revenue' #@param {type: "string"}
PROMPT_STRING = "Please parse these search results of financial data of the last 3 years and combine them into a tab delimited table: {results} " #@param {type: "string"}

# Combine the LLM with a prompt to make a simple chain
prompt = PromptTemplate(input_variables=['results'],
                        template=PROMPT_STRING)

chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

# Combine this chain with Enterprise Search in a new chain
es_chain = EnterpriseSearchChain(project=GCP_PROJECT,
                                 search_engine=SEARCH_ENGINE,
                                 chain=chain)

result = es_chain.run(SEARCH_QUERY)

result.split('\n')



> Entering new LLMChain chain...
Prompt after formatting:
Please parse these search results of financial data of the last 3 years and combine them into a tab delimited table: ['Total revenue increased by 4.0% to €45.6 billion. Group operating profit increased by €0.6 billion to €5.7 billion and basic earnings per share increased to\xa0...', 'May 18, 2021 ... Total revenue. €43.8bn. Strategic report. Governance. Financials. Other information. 2. Vodafone Group Plc. Annual Report 2021\xa0...', 'May 17, 2022 ... Total revenue increased by 4% to €45.6 billion, with Group organic service revenue growing by 2.6% this year. This was driven by consistent\xa0...', 'May 8, 2019 ... Although we met our financial guidance, our revenue growth slowed during the year and 5G spectrum auction costs were high, reducing our\xa0...', 'Nov 15, 2022 ... Total revenue increased by 2.0% to €22.9 billion (FY22 H1: €22.5 billion), as service revenue growth and higher equipment sales was partly\xa0...', 'May 1

['Year | Revenue | Operating Profit | Basic Earnings Per Share',
 '------- | -------- | -------------- | ------------------------',
 '2023 | €45.6 billion | €5.7 billion | €0.25',
 '2022 | €43.8 billion | €5.1 billion | €0.23',
 '2021 | €45.0 billion | €4.5 billion | €0.21']

In [ ]:
#@title Example - Answer a question about a search query
SEARCH_QUERY = 'Revenue' #@param {type: "string"}
YEAR = '2021 and 2022' #@param {type:"string"}

PROMPT_STRING = """
Please parse these search results and summarize them to answer
the following question.
Results:
{results}

Question:'What was the total amount of revenue in """ + YEAR + """
Answer:
"""

# Combine the LLM with a prompt to make a simple chain
prompt = PromptTemplate(input_variables=['results'],
                        template=PROMPT_STRING)

chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

# Combine this chain with Enterprise Search in a new chain
# This chain simply combines a Langchain LLMChain with our EnterpriseSearchRetriever
es_chain = EnterpriseSearchChain(project=GCP_PROJECT,
                                 search_engine=SEARCH_ENGINE,
                                 chain=chain)

result = es_chain.run(SEARCH_QUERY)

result.split('\n')



> Entering new LLMChain chain...
Prompt after formatting:

Please parse these search results and summarize them to answer 
the following question.
Results:
['May 16, 2023 ... The slowdown in quarterly trends was driven by lower MVNO revenues. Mobile service revenue grew by 8.0%* (Q3: 8.1%*, Q4: 2.8%*), driven by our\xa0...', 'May 18, 2021 ... Stable service revenue in FY21, returning to growth in the second half ... to grow revenues and cash flow over the medium-term.', "Service revenue relating to the provision of mobile services. Mobile termination rate ('MTR'), A per minute charge paid by a telecommunications network operator\xa0...", 'Total revenue increased by 4.0% to €45.6 billion. Group operating profit increased by €0.6 billion to €5.7 billion and basic earnings per share increased to\xa0...', 'Jul 23, 2021 ... Strong service revenue trend, including one-off growth of around 1.0 percentage point ... Vodafone Business service revenue growth of 2.7%*,\xa0...', 'Nov 15, 2022 ...

['The total revenue in 2021 was €45.6 billion and in 2022 it was €47.2 billion.']

# Pattern 2 - Use an LLM to produce queries, search, and summarize
_In some cases a user query might be too complex or abstract to be easily retrievable using a search engine. In this example we take the following approach:_
* _Take a complex query from the user_
* _Use an LLM to divide it into simple search terms_
* _Run a search for each query, retrieve and combine the results_
* _Ask the LLM to summarize the results in order to answer the query_

_The dataset in this example is an unstructured search engine containing a set of PDFs downloaded from [Worldbank](https://www.worldbank.org/en/home)_

In [ ]:
#@markdown ## Choose a search engine and define a complex query
COMPLEX_QUERY = 'What are the voice and data packages available for roaming in Europe?' #@param {"type": "string"}

# Initialise an Enterprise Search Retriever
retriever = EnterpriseSearchRetriever(GCP_PROJECT, SEARCH_ENGINE)

prompt = PromptTemplate(input_variables=["complex_query"], template="""Extract the most specific search terms from the following query:

Query:
'{complex_query}'

Search Terms:
* """)

#@markdown ## Fetch results from the LLM
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
terms = chain.run(COMPLEX_QUERY)

#@markdown ```
#@markdown * "revenue"
#@markdown * "budget"
#@markdown * "travel"
#@markdown ```

terms



> Entering new LLMChain chain...
Prompt after formatting:
Extract the most specific search terms from the following query:

Query:
'What are the voice and data packages available for roaming in Europe?'

Search Terms:
* 

> Finished chain.


'data\n*  roaming\n*  voice'

In [ ]:
#@title Clean up the response
#@markdown ## Select suitable terms and clean up unneccessary characters
lines_to_ignore = 0 #@param {"type": "integer"}
max_terms_to_search = 4 #@param {"type": "integer"}

clean_terms = [re.sub('[^\d\w\s]', '', q).strip() for q in terms.split('\n')[lines_to_ignore:lines_to_ignore + max_terms_to_search]]

#@markdown `['disclosure', 'appraisal', 'consultation', 'draft', 'sep']`
clean_terms

['data', 'roaming', 'voice']

In [ ]:
#@markdown ## Search each term and keep the top `n` results
num_results = 2 #@param {"type": "integer"}

results = []
for q in clean_terms:
  snippets = retriever.get_relevant_snippets(q)
  results.extend([s for s in snippets[:num_results]])

results = list(set(results)) # Deduplicate to keep prompt length down
results

['Our leadership in worldwide connectivity and coverage is supported by our long-term partnership with Vodafone Voice and Roaming Services, which helps us bring\xa0...',
 'Our international voice service is dedicated to the requirements of our 500+ million retail customers, spanning local markets across 26 countries.',
 "Johan Wibergh, Vodafone Group's chief technology officer (CTO) talks about Vodafone Neuron – the company's new cloud platform. This data ocean allows for\xa0...",
 'These Data Protection Terms are effective from 19 March 2019. Data Protection - When Service Terms identify Vodafone is. Data Controller.',
 'Learn how to establish roaming agreements with many operators through one hub connection and have those relationships managed centrally.',
 'Available to our carrier partners, enabling the highest quality voice termination to any destination.']

In [ ]:
#@markdown ## Combine the search results into an answer using an LLM
# Combine the LLM with a prompt to make a simple chain
prompt = PromptTemplate(input_variables=['query', 'results'],
                        template="""Please summarize the following contextual data to answer the following question. Provide references to the context in your answer:
Question: {query}
Context:
{results}
Answer with citations:""")
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

summary = chain.run({"query": COMPLEX_QUERY, "results": results})

summary.split('\n')



> Entering new LLMChain chain...
Prompt after formatting:
Please summarize the following contextual data to answer the following question. Provide references to the context in your answer:
Question: What are the voice and data packages available for roaming in Europe?
Context:
['Our leadership in worldwide connectivity and coverage is supported by our long-term partnership with Vodafone Voice and Roaming Services, which helps us bring\xa0...', 'Our international voice service is dedicated to the requirements of our 500+ million retail customers, spanning local markets across 26 countries.', "Johan Wibergh, Vodafone Group's chief technology officer (CTO) talks about Vodafone Neuron – the company's new cloud platform. This data ocean allows for\xa0...", 'These Data Protection Terms are effective from 19 March 2019. Data Protection - When Service Terms identify Vodafone is. Data Controller.', 'Learn how to establish roaming agreements with many operators through one hub connection and h

['Vodafone offers a variety of voice and data packages for roaming in Europe. These packages include:',
 '',
 '* The Vodafone Global Roaming Pass, which offers unlimited calls, texts, and data for a set price per day.',
 '* The Vodafone Europe Roaming Pass, which offers unlimited calls and texts, and a limited amount of data for a set price per day.',
 '* The Vodafone Europe Data Pass, which offers a set amount of data for a set price per day.',
 '',
 "For more information on Vodafone's roaming packages, please visit their website."]

# Pattern 3 - Using langchain's built in 'question answering' chain types
_Langchain provides some more sophisticated examples of chains which are designed specifically for question answering on your own documents. There are a few approaches, one of which is the [`refine`](https://docs.langchain.com/docs/components/chains/index_related_chains#refine) pattern._

_The refine chain is passed a set of langchain `Documents` and a query. It begins with the first document and sees if it can answer the question using the context. It then iteratively incorporates each subsequent document to refine its answer._

_In this example we convert a set of Enterprise Search snippets into `Documents` and pass them to the chain._

_We will use the same search engine and terms extracted from the previous example_

_[More examples in langchain docs here](https://python.langchain.com/en/latest/modules/chains/index_examples/qa_with_sources.html)_

In [ ]:
#@title We are using the same search engine, terms and question as the previous example


## Here is the function being used to convert search results to documents
 ## (already encoded in the `EnterpriseSearchRetriever` class)

# def search_response_to_documents(res) -> List[Document]:
#     """Retrieve langchain Documents from a search response"""
#     documents = []
#     for result in res.results:
#         data = MessageToDict(result.document._pb)
#         metadata = data.copy()
#         del metadata['derivedStructData']
#         del metadata['structData']
#         if data.get('derivedStructData') is None:
#             content = json.dumps(data.get('structData', {}))
#         else:
#             content = json.dumps([d.get('snippet') for d in data.get('derivedStructData', {}).get('snippets', []) if d.get('snippet') is not None])
#         documents.append(Document(page_content=content, metadata=metadata))
#     return documents

#@markdown ## Search for each search term and extract into a langchain `Document` format
#@markdown * This format just contains the snippets as `page_content` and the document title and link as `metadata`

document_responses = []
for t in clean_terms:
    document_responses.append(retriever.get_relevant_documents(t))

# This chain will run one LLM call for every document, so we likely do not want to keep all of the context if the document count is very large
for idx, d in enumerate(document_responses):
  print(f"Search {idx + 1}: {len(d)} documents")

Search 1: 10 documents
Search 2: 10 documents
Search 3: 10 documents


In [ ]:
# There are 65 documents total, so we will just keep the top 3 from each search
final_documents = [d for r in document_responses for d in r[:3]]

len(final_documents)

9

In [ ]:
#@title Load and run the chain using the documents
#@markdown **Reminder**: the question is 'What are the voice and data packages available for roaming in Europe?'

#@markdown Here we are printing the intermediate steps so you can see the end to end process

chain = load_qa_chain(llm, chain_type="refine", return_refine_steps=True)

chain({"input_documents": final_documents, "question": COMPLEX_QUERY}, return_only_outputs=True)

{'intermediate_steps': ['The context information is about the Data Protection Terms of Vodafone. It does not mention anything about Covid-19 or Vodafone revenues. So we cannot answer the question.',
  'The context information is about Vodafone Neuron, a new cloud platform. It does not mention anything about Covid-19 or Vodafone revenues. So we cannot answer the question.',
  'The context information is about Vodafone Red for Global Enterprise, a new plan for enterprises. It does not mention anything about Covid-19 or Vodafone revenues. So we cannot answer the question.',
  'The context information is about Vodafone Red for Global Enterprise, a new plan for enterprises. It does not mention anything about Covid-19 or Vodafone revenues. So we cannot answer the question.',
  'The context information is about Vodafone Red for Global Enterprise, a new plan for enterprises. It does not mention anything about Covid-19 or Vodafone revenues. So we cannot answer the question.',
  'The context inf

# Pattern 4 - Implement a Reasoning/Acting (ReAct) agent using langchain
_[ArXiv Paper](https://arxiv.org/pdf/2210.03629.pdf)_

_One of the more sophisticated workflows using LLMs is to create an 'agent' that can create new prompts for itself and then answer them in order to complete more complex tasks._

_One of the most powerful examples is the 'ReAct' (Reasoning + Acting) agent, which alternates between retrieving results from a prompt and assessing them in the context of a task. The agent autonomously determines if it has successfully completed the task and whether to continue answering new prompts or to return a result to the user._

_ReAct agents can be provided with an array of tools, each with a description. (These tools can be as simple as any python function that provides a string input and string output.) The ReAct agent uses the description of each tool to determine which to use at each stage._

_The following examples use Enterprise Search as a tool to retrieve a set of search result snippets to inform the prompt._

#### Use Cases
* Answering queries with complex intent
* Combining information retrieval with other tools such as data processing, mathematical operations, web search, etc.

### Step 1 - Create tools
In the **Imports and classes** section at the beginning of this notebook we defined a custom `EnterpriseSearchRetriever` class. This class exposes methods to retrieve search snippets and Documents. We pass these methods as 'tools' for the ReAct agent to use. Currently we have:

* `get_relevant_snippets` and `get_relevant_documents`:
  * Return a `List[str]` or `List[Document]` respectively of search results for a given search query

We also define a function later to use an LLM to split a complex query into multiple search terms, as we did in the previous pattern
* `extract_search_terms`:
  * Return a `List[str]` of search keywords for a given complex query
  * This uses the same process defined above to ask an LLM to split a complex query into simple keywords

In [ ]:
#@title ### Step 2 - Initialize a Vertex LLM and 'out of the box' langchain Agent
#@markdown _In this example we use the default `ZERO_SHOT_REACT_DESCRIPTION` agent that comes with Langchain. This provides a suitable prompt for the agent out of the box. The names and descriptions of provided tools are used to determine which is best to use._
# Initialize an Enterprise Search Retriever
retriever = EnterpriseSearchRetriever(GCP_PROJECT, SEARCH_ENGINE)

# The name and description here are critical in helping the LLM determine which tool to use
tools = [
    Tool.from_function(
        func=retriever.get_relevant_snippets,
        name = "Enterprise Search",
        description="""Search for a query."""
    )
]

# Combine the LLM with the search tool to make a ReAct agent
react_agent = initialize_agent(tools,
                               llm,
                               agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                               verbose=True)


COMPLEX_QUERY = 'What are the data packages available in Europe in 2023?' #@param {"type": "string"}
react_agent.run(COMPLEX_QUERY)




> Entering new AgentExecutor chain...
I need to find out what data packages are available in Europe in 2023
Action: Enterprise Search
Action Input: data packages available in Europe in 2023
Observation: ['Alternatively, frequent travellers can select to take a price plan which includes data roaming within their monthly domestic package. Both offers are available\xa0...', 'We provide 5G services commercially European countries and will continue to expand that ... connected and sharing data through a single super-fast network.', 'Results, reports & presentations ; May 22 2023. Simplified Vodafone holding structure ; May 16 2023. Full Year. Download ; Apr 18 2023. Social Contract ; Feb 01\xa0...', 'We are focused on growing our converged connectivity markets in Europe, and mobile data and payments in Africa. The next phase of our strategy focuses on\xa0...', 'Vodafone is a leading technology communications company in Europe and Africa, keeping society connected and building a digital fu

'Vodafone UK has launched a new mobile data plan that gives customers unlimited 5G data, minutes and texts for just £10 a month.'

### Step 3 - Adding memory to enable conversations.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import tool

conversational_memory = ConversationBufferMemory(
    memory_key='chat_history',
)

@tool
def enterprise_search(query: str):
    """
    Use this tool to find information in documents.
    Args:
    query: Analyse the conversation to come up with the parameter. This is the current problem the user is trying to solve.
    """
    return retriever.get_relevant_snippets(query)


react_agent = initialize_agent(llm=llm,
                               tools=[enterprise_search],
                               memory=conversational_memory,
                               agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                               verbose=True)


react_agent.run("What was the revenue of Vodafone in 2022?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: enterprise_search
Action Input: Vodafone revenue 2022
Observation: ['Vodafone has today announced results for the year ended 31 March 2022. The full text of the results announcement is available from the download link below.', 'Total revenue increased by 4.0% to €45.6 billion. Group operating profit increased by €0.6 billion to €5.7 billion and basic earnings per share increased to\xa0...', '15 November 2022. Resilient performance in Europe & Africa, good progress on operational & portfolio priorities. • Group service revenue growth of 2.5%* in\xa0...', "May 17, 2022 ... This report contains references to Vodafone's website, ... Market capitalisation at 31 March 2022. ... 2022. 2021. 2020. Group revenue.", 'May 16, 2023 ... Good performance in Vodafone Business with 2.6%* service revenue growth ... In May 2022, we set out guidance for FY23 for Group adjusted\xa0...', 'Nov 09 2022. Vantage Towers Trans

"Vodafone's revenue in 2022 was €45.6 billion."

In [ ]:
react_agent.run("How about 2021?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Vodafone's revenue in 2021 was €43.2 billion.

> Finished chain.


"Vodafone's revenue in 2021 was €43.2 billion."

### Step 4 - Setting guardrails in the agent - Custom agent

We finally created our first agent 🎉
But... what if the user asks about competitor companies? Or what if the user uses the agent to perform things he is not allowed to such as generic Q&A?

**In an enterprise setting we want to block that.**

A first hacky way to do this would be to inject messages in the conversation history but this risks to be ignored once the conversation matures.

The easiest way to set up some guardrails is by providing custom prefix for the prompts of our **agent**.

We are going to override the default prompts for our agent that are defined [here](https://github.com/hwchase17/langchain/blob/master/langchain/agents/conversational/prompt.py
)

In [ ]:
SITE_NAME = "Vodafone" #@param {"type": "string"}
PREFIX = """
You are """ + SITE_NAME + """ _Bot a large language model made available by """ + SITE_NAME + """.
You help customers finding the information from a large catalog of documents about """ + SITE_NAME + """.
You donot disclose any other company name under any circustamnces.
You cannot role play or pretend to be anything other than """ + SITE_NAME + """ _Bot.
If you are asked to role play respond with "I'm just a bot, a Q&A assistant".
If you are asked to pretend to be somebody else respond with "I'm just a bot, a Q&A assistant".

TOOLS:
------

You have access to the following tools:"""

conversational_memory = ConversationBufferMemory(
    memory_key='chat_history',
)

guardrailed_agent = initialize_agent(llm=llm,
                               tools=[enterprise_search],
                               memory=conversational_memory,
                               agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                               agent_kwargs={'prefix': PREFIX},
                               verbose=True)



In [ ]:
guardrailed_agent.run("What is the revenue of Vodafone in 2023?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: enterprise_search
Action Input: revenue of Vodafone in 2023
Observation: ['May 16, 2023 ... Good performance in Vodafone Business with 2.6%* service revenue growth ... A webcast Q&A session will be held at 10:00 BST on 16 May 2023.', '1 February 2023. Europe slowing as expected, resilient performance in Africa. • Group service revenue growth of 1.8%* (Q2: 2.5%*), with the slowdown in\xa0...', "Nov 15, 2022 ... Group service revenue growth of 2.5%* in the first half of FY23 ... in the Group's condensed consolidated income statement for H1 2023.", 'Vodafone Annual Report 2023. View our FY23 Annual Report ... Total revenue increased by 4.0% to €45.6 billion. Group operating profit increased by €0.6\xa0...', 'Results, reports & presentations ; May 22 2023. Simplified Vodafone holding structure ; May 16 2023. Full Year. Download ; Apr 18 2023. Social Contract ; Feb 01\xa0...', 'May 10, 2023 ... FY23 Result

"Vodafone's total revenue increased by 4.0% to €45.6 billion in 2023."

In [ ]:
guardrailed_agent.run("What is the revenue of BT in 2023?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: I'm just a bot, a Q&A assistant. I can't answer questions about other companies.

> Finished chain.


"I'm just a bot, a Q&A assistant. I can't answer questions about other companies."